In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

In [3]:
config_file = "configs/mnist.yaml"

cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2021-03-18 15:58:54,563 SSD INFO: Loaded configuration file configs/mnist.yaml
2021-03-18 15:58:54,565 SSD INFO: 
MODEL:
    NUM_CLASSES: 11
    BACKBONE:
        NAME: 'basic'
        PRETRAINED: False
        OUT_CHANNELS: [128, 256, 128, 128, 64, 64]
        INPUT_CHANNELS: 3
INPUT:
    IMAGE_SIZE: [300, 300]
DATASETS:
    TRAIN: ("mnist_detection_train", "mnist_detection_val")
    TEST: ("mnist_detection_val", )
SOLVER:
    MAX_ITER: 10000
    GAMMA: 0.1
    BATCH_SIZE: 16
    LR: 2e-3
OUTPUT_DIR: 'outputs/basic'
DATASET_DIR: "datasets"
2021-03-18 15:58:54,566 SSD INFO: Running with config:
DATASETS:
  TEST: ('mnist_detection_val',)
  TRAIN: ('mnist_detection_train', 'mnist_detection_val')
DATASET_DIR: datasets
DATA_LOADER:
  NUM_WORKERS: 4
  PIN_MEMORY: True
EVAL_STEP: 500
INPUT:
  IMAGE_SIZE: [300, 300]
  PIXEL_MEAN: [123.675, 116.28, 103.53]
  PIXEL_STD: [1, 1, 1]
LOG_STEP: 10
MODEL:
  BACKBONE:
    INPUT_CHANNELS: 3
    NAME: basic
    OUT_CHANNELS: (128, 256, 128, 128, 64, 64)

In [4]:
model = start_train(cfg)

Detector initialized. Total Number of params:  2.68M
Backbone number of parameters: 2.12M
SSD Head number of parameters: 553.4K
2021-03-18 15:59:02,660 SSD.trainer INFO: No checkpoint found.


Generating dataset, saving to: datasets/mnist_detection/train: 100%|██████████| 10000/10000 [00:37<00:00, 269.49it/s]
Generating dataset, saving to: datasets/mnist_detection/test: 100%|██████████| 1000/1000 [00:03<00:00, 265.96it/s]


2021-03-18 15:59:46,052 SSD.trainer INFO: Start training ...


NameError: name 'out_channel' is not defined

In [ ]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)